<a href="https://colab.research.google.com/github/sameep114/Transformer-networks-for-abstractive-summarization/blob/master/summarizer_latest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install -q --upgrade ipython
# !pip install -q --upgrade ipykernel

In [ ]:
import re
import os
import time
import numpy as np
import pandas as pd 
import unicodedata
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy, CategoricalCrossentropy
from tensorflow.keras.preprocessing.text import Tokenizer 
from tensorflow.keras.optimizers.schedules import LearningRateSchedule
import csv


In [ ]:
BUFFER_SIZE = 20000
BATCH_SIZE = 64
embedding_dim=300

In [ ]:
from google.colab import drive
drive.mount("/content/drive")
# drive.flush_and_unmount()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Loading Data

In [ ]:
news = pd.read_csv("/content/drive/MyDrive/Datasets/news_summary_more-1 - news_summary_more-1 translated.csv", delimiter=",",encoding="UTF8")

In [ ]:
#  news.drop(['headlines','text'], axis=1, inplace=True)

In [ ]:
 news=news[news.index < 100]

In [ ]:
news.tail(3)

,headlines,text,HEADLINES,CTEXT
97,Decide about an alliance within 15 days: Uddha...,Shiv Sena chief Uddhav Thackeray has asked the...,भाजपा गर्न Uddhav: 15 दिन भित्र एउटा सम्बन्ध ब...,शिवसेना प्रमुख Uddhav ठाकरे 15 दिन भित्रमा शिव...
98,"Grand alliance, if wins, will see a new PM eve...",BJP President Amit Shah on Wednesday took a di...,"महान् सम्बन्ध, WINS भने, हरेक दिन नयाँ PM देख्...","बुधबार भाजपा अध्यक्ष अमित शाह "", प्रत्येक विरो..."
99,Demonetisation made homes affordable for youth...,Prime Minister Narendra Modi on Wednesday said...,Demonetisation गरे घरमा युवा को लागि किफायती प...,बुधबार प्रधानमन्त्री नरेन्द्र मोदी आफ्नो सरकार...


In [ ]:
# shuffle garda random value aaidiyera doc val ma kin value rakhya nai tha vayena so remove this
from sklearn.utils import shuffle
# shuffling the data 
news = shuffle(news)
news.head()



,headlines,text,HEADLINES,CTEXT
53,OnePlus named bestselling premium smartphone b...,OnePlus has emerged as India's best-selling pr...,OnePlus 2018 को प्रिमियम स्मार्टफोन ब्रान्ड बि...,"OnePlus मुकाबला गरेर रिपोर्ट प्रति रूपमा, 2018..."
1,Delhi techie wins free food from Swiggy for on...,Kunal Shah's credit card bill payment platform...,दिल्ली प्रौद्योगिकी विशेषज्ञ Cred एक वर्षको ला...,"Kunal शाह क्रेडिट कार्ड बिल भुक्तानी मंच, Cred..."
60,"Govt to present Interim Budget, not full Budge...",The Finance Ministry on Wednesday said the gov...,"फेब्रुअरी 1 मा उपस्थित अन्तरिम बजेट, पूर्ण बजे...",बुधवार मा वित्त मन्त्रालयले स्पष्टीकरण पूर्ण ब...
23,Isha Ambani features on February cover of Vogu...,Reliance Industries' Chairman Mukesh Ambani's ...,ईशा अंबानी वोग पत्रिका को फेब्रुअरी आवरण मा वि...,रिलायन्स इंडस्ट्रीज 'अध्यक्ष मुकेश अंबानी छोरी...
65,Qatar to invest $200 million in Airtel Africa,Airtel on Wednesday said that Qatar's sovereig...,एयरटेल अफ्रीका लगानी गर्न कतार लाख $ 200,बुधबार एयरटेल कतार गरेको सार्वभौम धन कोष कतार ...


In [ ]:

document,summary= pd.DataFrame(), pd.DataFrame()
document['CTEXT'] = news['CTEXT']
summary['HEADLINES'] = news['HEADLINES']

document.shape,summary.shape

((100, 1), (100, 1))

In [ ]:
# replacing many abbreviations and lower casing the words
def clean_words(sentence):
    sentence = str(sentence).lower()
    # sentence = unicodedata.normalize('NFKD', sentence).encode('ascii', 'ignore').decode('utf-8', 'ignore') # for converting é to e and other accented chars
    sentence = re.sub(r"\"", "", sentence)
    sentence = re.sub(r"\'", "", sentence)
    sentence = re.sub(r"[\[\]\\()\"$#%/@;:<>{}`+=~|.!?,-]", "", sentence)
    sentence = re.sub(r"&", "", sentence)
    sentence = re.sub(r"\\n", "", sentence)
    sentence = sentence.strip()
    return sentence 

In [ ]:
summary['HEADLINES'] = summary['HEADLINES'].apply(lambda x: clean_words(x))
document['CTEXT'] = document['CTEXT'].apply(lambda x: clean_words(x))


### Preprocessing

In [ ]:
# for decoder sequence
start_token, end_token = '<startseq>' , '<endseq>'
summary = summary.apply(lambda x: start_token + ' ' + x + ' ' + end_token)
summary.head()

,HEADLINES
53,<startseq> oneplus 2018 को प्रिमियम स्मार्टफोन...
1,<startseq> दिल्ली प्रौद्योगिकी विशेषज्ञ cred ए...
60,<startseq> फेब्रुअरी 1 मा उपस्थित अन्तरिम बजेट...
23,<startseq> ईशा अंबानी वोग पत्रिका को फेब्रुअरी...
65,<startseq> एयरटेल अफ्रीका लगानी गर्न कतार लाख ...


In [ ]:
val_split=0.1
#train validation split
summary_train = summary[int(len(summary)*val_split):]
summary_val = summary[:int(len(summary)*val_split)]
document_train = document[int(len(summary)*val_split):]
document_val = document[:int(len(summary)*val_split)]

len(document_val),len(document_train)

(10, 90)

In [ ]:
document_train.iloc[0], summary_train.iloc[0]

(CTEXT    aegon जीवन iterm बीमा योजना संग ग्राहकहरु आफ्न...
 Name: 3, dtype: object,
 HEADLINES    <startseq> aegon जीवन iterm बीमा योजना कर सेव ...
 Name: 3, dtype: object)

FINDING MAX LENGTH OF TEXT AND SUMMARY

In [ ]:

# because there are senteces with unusually long lengths, 
# we caculate the max length that 95% of sentences are shorter than that
def max_length(shorts, longs, prct):
    # Create a list of all the captions
    
    length_longs = list(len(d.split()) for d in longs)
    length_shorts = list(len(d.split()) for d in shorts)

    print('percentile {} of length of news: {}'.format(prct,np.percentile(length_longs, prct)))
    print('longest sentence: ', max(length_longs))
    print()
    print('percentile {} of length of summaries: {}'.format(prct,np.percentile(length_shorts, prct)))
    print('longest sentence: ', max(length_shorts))
    print()
    return int(np.percentile(length_longs, prct)),int(np.percentile(length_shorts, prct))

# selecting sentence length based on the percentile of data that fits in the length
max_len_news, max_len_summary= max_length(summary_train['HEADLINES'].to_list(), document_train['CTEXT'].to_list(), 90)


print('max-length document chosen for training: ', max_len_news)
print('max-length summaries chosen for training: ', max_len_summary)


percentile 90 of length of news: 60.0
longest sentence:  69

percentile 90 of length of summaries: 14.0
longest sentence:  17

max-length document chosen for training:  60
max-length summaries chosen for training:  14


Dataset Preparation

In [ ]:
def create_vocab(shorts, longs = None, minimum_repeat = 3):

    # Create a list of all the captions
    all_captions = []
    for s in shorts:
        all_captions.append(s)

    # Consider only words which occur at least minimum_occurrence times in the corpus
    word_counts = {}
    nsents = 0
    for sent in all_captions:
        nsents += 1
        for w in sent.split(' '):
            word_counts[w] = word_counts.get(w, 0) + 1

    vocab = [w for w in word_counts if word_counts[w] >= minimum_repeat]
    
    vocab = list(set(vocab))
    return vocab

In [ ]:

# each word in the vocabulary must be used in the data atleast minimum_repeat times
vocab_dec = create_vocab(summary_train['HEADLINES'].to_list(), minimum_repeat=3) # here we just use the words in vocabulary of summaries
# removing one character words from vocab except for 'a'
# for v in vocab_dec:
#     if len(v) == 1 and v!='अ' and v!='क':
#         vocab_dec.remove(v) 
        
vocab_dec = sorted(vocab_dec)[1:] # [1:] is for the '' 
vocab_dec[:10]

['<endseq>',
 '<startseq>',
 'pm',
 'अभिनेत्री',
 'अमेरिकी',
 'आफ्नो',
 'आरोप',
 'एक',
 'काम',
 'को']

In [ ]:

# each word in the vocabulary must be used in the data atleast minimum_repeat times
vocab_enc = create_vocab(document_train['CTEXT'].to_list(), minimum_repeat=3) # here we just use the words in vocabulary of summaries
# removing one character words from vocab except for 'a'
# for v in vocab_enc:
#     if len(v) == 1 and v!='a' and v!='i':
#         vocab_enc.remove(v) 
        
vocab_enc = sorted(vocab_enc)[1:] # [1:] is for the '' 
vocab_enc[:10]

['10',
 '12',
 '2018',
 '22',
 '90',
 'brawl',
 'commode',
 'eufs',
 'filmmaker',
 'hirani']

In [ ]:
oov_token = '<UNK>'
filters = '!"#$%&()*+,-./:;=?@[\\]^_`{|}~\t\n' # making sure all the last non digit non alphabet chars are removed
document_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters = filters,oov_token=oov_token)
summary_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters = filters,oov_token=oov_token)
document_tokenizer.fit_on_texts(vocab_enc)
summary_tokenizer.fit_on_texts(vocab_dec)#summaries_train['short'])

# caculating number of words in vocabulary of encoder and decoder
# they are important for positional encoding
encoder_vocab_size = len(document_tokenizer.word_index) + 1 
decoder_vocab_size = len(summary_tokenizer.word_index) + 1

# vocab_size
encoder_vocab_size, decoder_vocab_size

(357, 40)

In [ ]:
ixtoword_enc = {} # index to word dic
ixtoword_dec = {} # index to word dic

wordtoix_enc = document_tokenizer.word_index # word to index dic
ixtoword_enc[0] = '<PAD0>' # no word in vocab has index 0. but padding is indicated with 0
ixtoword_dec[0] = '<PAD0>' # no word in vocab has index 0. but padding is indicated with 0

for w in document_tokenizer.word_index:
    ixtoword_enc[document_tokenizer.word_index[w]] = w
################################################
wordtoix_dec = summary_tokenizer.word_index # word to index dic

for w in summary_tokenizer.word_index:
    ixtoword_dec[summary_tokenizer.word_index[w]] = w


#### Tokenizing the texts into integer tokens

In [ ]:
# # since < and > from default tokens cannot be removed
# filters = '!"#$%&()*+,-./:;=?@[\\]^_`{|}~\t\n'
# oov_token = '<unk>'

In [ ]:
# document_tokenizer = tf.keras.preprocessing.text.Tokenizer(oov_token=oov_token)
# summary_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters=filters, oov_token=oov_token)

In [ ]:

# document_tokenizer.fit_on_texts(document)
# summary_tokenizer.fit_on_texts(summary)

In [ ]:
inputs = document_tokenizer.texts_to_sequences(document_train['CTEXT'])
targets = summary_tokenizer.texts_to_sequences(summary_train['HEADLINES'])

inputs_val= document_tokenizer.texts_to_sequences(document_val['CTEXT'])
targets_val= document_tokenizer.texts_to_sequences(summary_val['HEADLINES'])



In [ ]:
summary_tokenizer.texts_to_sequences(["‘लभ स्टेसन’ भियतनाम र इन्डोनेसियामा"])

[[1, 1, 1, 1, 1]]

#yaha bata tala ko bert tokenizer use gardeko ho try garna ko lagi

In [ ]:
# # since * and ~ from default tokens cannot be removed
# filters = '!"#$%&()+,-./:;=?@[\\]^_`{|}\t\n'

In [ ]:
# !pip install transformers
# from transformers import BertTokenizer

# # load bert tokenizer
# print("loading BERT  tokenizer...")
# document_tokenizer=BertTokenizer.from_pretrained('bert-base-multilingual-cased',do_lower=True)
# summary_tokenizer=BertTokenizer.from_pretrained('bert-base-multilingual-cased',do_lower=True, filters=filters)

In [ ]:
# sent=("सहज रुपमा भन्नुपर्दा नेपाली भनेको नेपालमा बस्ने नागरिक हो । हामी नेपाली नेपाललाई अंग्रेजबाट मुक्त गराएर स्वच्छ बनाउने बिर योद्धा हो ।")
# decoder_input=(summary_tokenizer.vocab_size)
# print(decoder_input)

# encodded=summary_tokenizer.encode(sent, add_special_tokens= True)
# print("encodded yesko tala")
# print(encodded)

# encoder_input = tf.expand_dims(encodded, 0)
# print(encoder_input)

# token=summary_tokenizer.tokenize(sent)
# print("\n yesko tala tokens")
# print(token)

# ids=summary_tokenizer.convert_tokens_to_ids(token)
# print("\n yesko tala ids")
# print(ids)

# tokens=summary_tokenizer.convert_ids_to_tokens(ids)
# print("\n yesko tala tokens")
# print(tokens)

# print("\n yesko tala string")
# print(summary_tokenizer.convert_tokens_to_string(tokens))


# print(summary_tokenizer.tokenize("CLS"))
# print(summary_tokenizer.convert_tokens_to_ids(
# summary_tokenizer.tokenize("~")))
# print(summary_tokenizer.convert_tokens_to_ids(
# summary_tokenizer.tokenize("*")))

In [ ]:
# val_split=0.1
# #train validation split
# summary_train = summary[int(len(summary)*val_split):]
# summary_val = summary[:int(len(summary)*val_split)]
# document_train = document[int(len(document)*val_split):]

# document_val = document[:int(len(document)*val_split)]

# len(document_val),len(document_train),len(summary_val),len(summary_train)

In [ ]:

# document_val.reset_index(inplace=True, drop=True)
# summary_val.reset_index(inplace=True, drop=True)
# document_train.reset_index(inplace=True, drop=True)
# summary_train.reset_index(inplace=True, drop=True)

In [ ]:
# # tokenize all of the  input sentences and map the tokens to their respective word ids
# inputs=[]

# #for every sentences
# for sent in document_train:
#   encodded_sent=document_tokenizer.encode(
#                     sent, add_special_tokens=True,
#                         )
#   inputs.append(encodded_sent)

# # print first paragraph now as list of ids.
# print('original:',document_train[0])
# print('token IDs',inputs[0])

# # tokenize all of the target sentences and map the tokens to their respective word ids
# targets=[]

# #for every sentences
# for sent in summary_train:
#   encodded_sent=summary_tokenizer.encode(
#                     sent, add_special_tokens=True,
#                         )
#   targets.append(encodded_sent)

# # print first paragraph now as list of ids.
# print('original:',summary_train[0])
# print('token IDs',targets[0])

# print(inputs)

In [ ]:
# # tokenize all of the  input sentences and map the tokens to their respective word ids
# inputs_val=[]

# #for every sentences
# for sent in document_val:
#   encodded_sent=document_tokenizer.encode(
#                     sent, add_special_tokens=True,
#                         )
#   inputs_val.append(encodded_sent)

# # print first paragraph now as list of ids.
# print('original:',document_val[0])
# print('token IDs',inputs_val[0])

# # tokenize all of the target sentences and map the tokens to their respective word ids
# targets_val=[]

# #for every sentences
# for sent in summary_val:
#   encodded_sent=summary_tokenizer.encode(
#                     sent, add_special_tokens=True,
#                         )
#   targets_val.append(encodded_sent)

# # print first paragraph now as list of ids.
# print('original:',summary_val[0])
# print('token IDs',targets_val[0])

# print(inputs)

In [ ]:
# encoder_vocab_size = len(inputs[]) +1
# decoder_vocab_size = len(targets[0]) +1

# # vocab_size
# encoder_vocab_size, decoder_vocab_size

In [ ]:
# # get encoder and decoder vocab size
# enc_greatest=[]
# for inp in inputs:
#   enc_greatest.append(max(inp))
# encoder_vocab_size=(max(enc_greatest))  +1



# # get encoder and decoder vocab size
# greatest=[]
# for inp in targets:
#   greatest.append(max(inp))
# decoder_vocab_size=max(greatest) + 1   


# # encoder_vocab_size=119548
# # decoder_vocab_size=119548

# encoder_vocab_size, decoder_vocab_size

In [ ]:
# summary_tokenizer.convert_ids_to_tokens([19,43,432,44,41])
# summary_tokenizer.convert_tokens_to_string(summary_tokenizer.convert_ids_to_tokens([13,43,432,4412,41]))

In [ ]:
# inputs = document_tokenizer.encode(document_train)
# targets = summary_tokenizer.encode(summary_train)

# inputs_val= document_tokenizer.encode(document_val)
# targets_val= document_tokenizer.encode(summary_val)


#yaha bhanda mathi ho bert tokenizer

In [ ]:
summary_tokenizer.sequences_to_texts([[13,43,432,4412,41]])

['छ <UNK> <UNK> <UNK> <UNK>']

In [ ]:
encoder_vocab_size = len(document_tokenizer.word_index) + 1
decoder_vocab_size = len(summary_tokenizer.word_index) + 1

# vocab_size
encoder_vocab_size, decoder_vocab_size

(357, 40)

#### Obtaining insights on lengths for defining maxlen

In [ ]:
document_lengths = pd.Series([len(x) for x in document])
summary_lengths = pd.Series([len(x) for x in summary])

In [ ]:
document_lengths.describe()

count    1.0
mean     5.0
std      NaN
min      5.0
25%      5.0
50%      5.0
75%      5.0
max      5.0
dtype: float64

In [ ]:
summary_lengths.describe()

count    1.0
mean     9.0
std      NaN
min      9.0
25%      9.0
50%      9.0
75%      9.0
max      9.0
dtype: float64

In [ ]:
# # maxlen
# # taking values > and round figured to 75th percentile
# # at the same time not leaving high variance/
# max_len_news=encoder_vocab_size+1
# max_len_summary=int(encoder_vocab_size/3)



#### Padding/Truncating sequences for identical sequence lengths

In [ ]:
inputs = tf.keras.preprocessing.sequence.pad_sequences(inputs, maxlen=max_len_news, padding='post', truncating='post')
targets = tf.keras.preprocessing.sequence.pad_sequences(targets, maxlen=max_len_summary, padding='post', truncating='post')

inputs_val = tf.keras.preprocessing.sequence.pad_sequences(inputs_val, maxlen=max_len_news, padding='post', truncating='post')
targets_val = tf.keras.preprocessing.sequence.pad_sequences(targets_val, maxlen=max_len_summary, padding='post', truncating='post')


### Creating dataset pipeline

In [ ]:
# inputs = tf.cast(inputs, dtype=tf.int32)
# targets = tf.cast(targets, dtype=tf.int32)

# inputs.shape

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((inputs, targets)).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
dataset_val = tf.data.Dataset.from_tensor_slices((inputs_val, targets_val)).shuffle(BUFFER_SIZE).batch(BATCH_SIZE*2)


In [ ]:
document_val.reset_index(inplace=True, drop=True)
summary_val.reset_index(inplace=True, drop=True)
document_train.reset_index(inplace=True, drop=True)
summary_train.reset_index(inplace=True, drop=True)

# defining the model

In [ ]:
# import matplotlib.pyplot as plt

# def hist(history):
#     plt.title('Loss')

#     x= [i[0] for i in history['val']]
#     y=[i[1] for i in history['val']]
#     plt.plot(x,y,'x-')
    
#     x= [i[0] for i in history['train']]
#     y=[i[1] for i in history['train']]    
#     plt.plot(x,y,'o-')

#     plt.legend(['validation','train'])
#     plt.show()
#     print('smallest val loss:', sorted(history['val'],key=lambda x: x[1])[0])

### Positional Encoding for adding notion of position among words as unlike RNN this is non-directional

In [ ]:
def get_angles(position, i, d_model):
    angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(d_model))
    return position * angle_rates

In [ ]:
def positional_encoding(position, d_model):
    angle_rads = get_angles(
        np.arange(position)[:, np.newaxis],
        np.arange(d_model)[np.newaxis, :],
        d_model
    )

    # apply sin to even indices in the array; 2i
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])

    # apply cos to odd indices in the array; 2i+1
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

    pos_encoding = angle_rads[np.newaxis, ...]

    return tf.cast(pos_encoding, dtype=tf.float32)


### Masking

- Padding mask for masking "pad" sequences
- Lookahead mask for masking future words from contributing in prediction of current words in self attention

In [ ]:
def create_padding_mask(seq):
    seq = tf.cast(tf.math.equal(seq, 0), tf.float32)
    return seq[:, tf.newaxis, tf.newaxis, :]

In [ ]:
def create_look_ahead_mask(size):
    mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
    return mask

### Building the Model

#### Scaled Dot Product

In [ ]:
def scaled_dot_product_attention(q, k, v, mask):
    matmul_qk = tf.matmul(q, k, transpose_b=True)

    dk = tf.cast(tf.shape(k)[-1], tf.float32)
    scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

    if mask is not None:
        scaled_attention_logits += (mask * -1e9)  

    attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)

    output = tf.matmul(attention_weights, v)
    return output, attention_weights

#### Multi-Headed Attention

In [ ]:

class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model

        assert d_model % self.num_heads == 0

        self.depth = d_model // self.num_heads

        # self.wq = tf.keras.layers.Dense(d_model)
        # self.wk = tf.keras.layers.Dense(d_model)
        # self.wv = tf.keras.layers.Dense(d_model)

        # self.dense = tf.keras.layers.Dense(d_model)

        self.wq = tf.keras.layers.Dense(d_model,kernel_regularizer=tf.keras.regularizers.l2(l=lmbda_l2))
        self.wk = tf.keras.layers.Dense(d_model,kernel_regularizer=tf.keras.regularizers.l2(l=lmbda_l2))
        self.wv = tf.keras.layers.Dense(d_model,kernel_regularizer=tf.keras.regularizers.l2(l=lmbda_l2))

        self.dense = tf.keras.layers.Dense(d_model,kernel_regularizer=tf.keras.regularizers.l2(l=lmbda_l2))
        
    def split_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(x, perm=[0, 2, 1, 3])
    
    def call(self, v, k, q, mask):
        batch_size = tf.shape(q)[0]

        q = self.wq(q)
        k = self.wk(k)
        v = self.wv(v)

        q = self.split_heads(q, batch_size)
        k = self.split_heads(k, batch_size)
        v = self.split_heads(v, batch_size)

        scaled_attention, attention_weights = scaled_dot_product_attention(
            q, k, v, mask)

        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])

        concat_attention = tf.reshape(scaled_attention, (batch_size, -1, self.d_model))
        output = self.dense(concat_attention)
            
        return output, attention_weights

# embeddings preparation

In [ ]:
# !wget http://nlp.stanford.edu/data/glove.6B.zip

--2021-02-20 09:03:51--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2021-02-20 09:03:52--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2021-02-20 09:03:52--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [ ]:
# !unzip glove*.zip

Archive:  glove.6B.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of glove.6B.zip or
        glove.6B.zip.zip, and cannot find glove.6B.zip.ZIP, period.


In [ ]:
# !ls
# !pwd

drive  glove.6B.zip  sample_data
/content


In [ ]:
# Making the embedding mtrix
def make_embedding_layer(vocab_len, wordtoix, embedding_dim=300, glove=True, glove_path= '../glove'):
    if glove == False:
        print('Just a zero matrix loaded')
        embedding_matrix = np.zeros((vocab_len, embedding_dim)) # just a zero matrix 
    else:
        print('Loading glove...')
        glove_dir = "/content/drive/MyDrive/Datasets/"
        embeddings_index = {} 
        fi = open(glove_dir +'nepali_embeddings_word2vec.txt', encoding="utf-8")
    
        for line in fi:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
        fi.close()
        # Get n-dim dense vector for each of the vocab_rocc
        embedding_matrix = np.zeros((vocab_len, embedding_dim)) # to import as weights for Keras Embedding layer
        for word, i in wordtoix.items():
            embedding_vector = embeddings_index.get(word)
            if embedding_vector is not None:
                # Words not found in the embedding index will be all zeros
                embedding_matrix[i] = embedding_vector
        
        print("GloVe ",embedding_dim, ' loaded!')

    embedding_layer = Embedding(vocab_len, embedding_dim, mask_zero=True, trainable=False) # we have a limited vocab so we 
                                                                                           # do not train the embedding layer
                                                                                           # we use 0 as padding so => mask_zero=True
    embedding_layer.build((None,))
    embedding_layer.set_weights([embedding_matrix])
    return embedding_layer

embeddings_encoder = make_embedding_layer(encoder_vocab_size, wordtoix_enc, embedding_dim=embedding_dim, glove=True)
embeddings_decoder = make_embedding_layer(decoder_vocab_size, wordtoix_dec, embedding_dim=embedding_dim, glove=True)

Loading glove...
GloVe  300  loaded!
Loading glove...
GloVe  300  loaded!


### Feed Forward Network

In [ ]:
# hyper-params
init_lr = 1e-3
lmbda_l2 = 0.1
d_out_rate = 0.1
num_layers = 4
d_model = embedding_dim # d_model is the representation dimension or embedding dimension of a word (usually in the range 128–512)
dff = 512 # number of neurons in feed forward network
num_heads = 5

In [ ]:
def point_wise_feed_forward_network(d_model, dff):
    return tf.keras.Sequential([
        tf.keras.layers.Dense(dff, activation='relu'),
        tf.keras.layers.Dense(d_model)
    ])

#### Fundamental Unit of Transformer encoder

In [ ]:
class EncoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, dff, rate=0.1):
        super(EncoderLayer, self).__init__()

        self.mha = MultiHeadAttention(d_model, num_heads)
        self.ffn = point_wise_feed_forward_network(d_model, dff)

        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)
    
    def call(self, x, training, mask):
        attn_output, _ = self.mha(x, x, x, mask)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(x + attn_output)

        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        out2 = self.layernorm2(out1 + ffn_output)

        return out2


#### Fundamental Unit of Transformer decoder

In [ ]:
class DecoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, dff, rate=0.1):
        super(DecoderLayer, self).__init__()

        self.mha1 = MultiHeadAttention(d_model, num_heads)
        self.mha2 = MultiHeadAttention(d_model, num_heads)

        self.ffn = point_wise_feed_forward_network(d_model, dff)

        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)
        self.dropout3 = tf.keras.layers.Dropout(rate)
    
    
    def call(self, x, enc_output, training, look_ahead_mask, padding_mask):
        attn1, attn_weights_block1 = self.mha1(x, x, x, look_ahead_mask)
        attn1 = self.dropout1(attn1, training=training)
        out1 = self.layernorm1(attn1 + x)

        attn2, attn_weights_block2 = self.mha2(enc_output, enc_output, out1, padding_mask)
        attn2 = self.dropout2(attn2, training=training)
        out2 = self.layernorm2(attn2 + out1)

        ffn_output = self.ffn(out2)
        ffn_output = self.dropout3(ffn_output, training=training)
        out3 = self.layernorm3(ffn_output + out2)

        return out3, attn_weights_block1, attn_weights_block2


#### Encoder consisting of multiple EncoderLayer(s)

In [ ]:
class Encoder(tf.keras.layers.Layer):
    def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size, maximum_position_encoding, rate=0.1):
        super(Encoder, self).__init__()

        self.d_model = d_model
        self.num_layers = num_layers

        self.embedding = tf.keras.layers.Embedding(input_vocab_size, d_model)
        self.pos_encoding = positional_encoding(maximum_position_encoding, self.d_model)

        self.enc_layers = [EncoderLayer(d_model, num_heads, dff, rate) for _ in range(num_layers)]

        self.dropout = tf.keras.layers.Dropout(rate)
        
    def call(self, x, training, mask):
        seq_len = tf.shape(x)[1]

        x = self.embedding(x)
        x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        x += self.pos_encoding[:, :seq_len, :]

        x = self.dropout(x, training=training)
    
        for i in range(self.num_layers):
            x = self.enc_layers[i](x, training, mask)
    
        return x


#### Decoder consisting of multiple DecoderLayer(s)

In [ ]:
class Decoder(tf.keras.layers.Layer):
    def __init__(self, num_layers, d_model, num_heads, dff, target_vocab_size, maximum_position_encoding, rate=0.1):
        super(Decoder, self).__init__()

        self.d_model = d_model
        self.num_layers = num_layers

        self.embedding = tf.keras.layers.Embedding(target_vocab_size, d_model)
        self.pos_encoding = positional_encoding(maximum_position_encoding, d_model)

        self.dec_layers = [DecoderLayer(d_model, num_heads, dff, rate) for _ in range(num_layers)]
        self.dropout = tf.keras.layers.Dropout(rate)
    
    def call(self, x, enc_output, training, look_ahead_mask, padding_mask):
        seq_len = tf.shape(x)[1]
        attention_weights = {}

        x = self.embedding(x)
        x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        x += self.pos_encoding[:, :seq_len, :]

        x = self.dropout(x, training=training)

        for i in range(self.num_layers):
            x, block1, block2 = self.dec_layers[i](x, enc_output, training, look_ahead_mask, padding_mask)

            attention_weights['decoder_layer{}_block1'.format(i+1)] = block1
            attention_weights['decoder_layer{}_block2'.format(i+1)] = block2
    
        return x, attention_weights


#### Finally, the Transformer

In [ ]:
class Transformer(tf.keras.Model):
    def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size, target_vocab_size, pe_input, pe_target, rate=0.15):
        super(Transformer, self).__init__()

        self.encoder = Encoder(num_layers, d_model, num_heads, dff, input_vocab_size, pe_input, rate)

        self.decoder = Decoder(num_layers, d_model, num_heads, dff, target_vocab_size, pe_target, rate)

        self.final_layer = tf.keras.layers.Dense(target_vocab_size)
    
    def call(self, inp, tar, training, enc_padding_mask, look_ahead_mask, dec_padding_mask):
        enc_output = self.encoder(inp, training, enc_padding_mask)

        dec_output, attention_weights = self.decoder(tar, enc_output, training, look_ahead_mask, dec_padding_mask)

        final_output = self.final_layer(dec_output)

        return final_output, attention_weights


### Training

In [ ]:


# EPOCHS = 20

#### Adam optimizer with custom learning rate scheduling

In [ ]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super(CustomSchedule, self).__init__()

        self.d_model = d_model
        self.d_model = tf.cast(self.d_model, tf.float32)

        self.warmup_steps = warmup_steps
    
    def __call__(self, step):
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps ** -1.5)

        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)


#### Defining losses and other metrics 

In [ ]:
# learning_rate = CustomSchedule(d_model)

# optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

In [ ]:

lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=init_lr, 
    decay_steps=4000, # approximately 5 epochs
    decay_rate=0.95) 

In [ ]:

optimizer2 = Adam(lr_schedule , beta_1=0.9, beta_2=0.98, epsilon=1e-9) # changed to init
loss_object = SparseCategoricalCrossentropy(from_logits=True, reduction='none') # added softmax changed from_logits to false

In [ ]:
def loss_function(real, pred, l2= False):
 
    if l2:
        lambda_ = 0.0001
        l2_norms = [tf.nn.l2_loss(v) for v in transformer.trainable_variables]
        l2_norm = tf.reduce_sum(l2_norms)
        l2_value = lambda_ * l2_norm
        loss_ = loss_object(real, pred) + l2_value
    else:
        loss_ = loss_object(real, pred) 
    
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask
    
    return tf.reduce_sum(loss_)/tf.reduce_sum(mask)

In [ ]:
train_loss = tf.keras.metrics.Mean(name='train_loss')

#### Transformer

In [ ]:
transformer = Transformer(
    num_layers, 
    d_model, 
    num_heads, 
    dff,
    encoder_vocab_size, 
    decoder_vocab_size, 
    pe_input=encoder_vocab_size, 
    pe_target=decoder_vocab_size,
)

#### Masks

In [ ]:
def create_masks(inp, tar):
    enc_padding_mask = create_padding_mask(inp)
    dec_padding_mask = create_padding_mask(inp)

    look_ahead_mask = create_look_ahead_mask(tf.shape(tar)[1])
    dec_target_padding_mask = create_padding_mask(tar)
    combined_mask = tf.maximum(dec_target_padding_mask, look_ahead_mask)
  
    return enc_padding_mask, combined_mask, dec_padding_mask


#### Checkpoints

In [ ]:
checkpoint_path4 ="/content/drive/MyDrive/checkpoints5"

ckpt4 = tf.train.Checkpoint(transformer=transformer, optimizer=optimizer2)

ckpt_manager4 = tf.train.CheckpointManager(ckpt4, checkpoint_path4, max_to_keep=100)

if ckpt_manager4.latest_checkpoint:
    ckpt4.restore(ckpt_manager4.latest_checkpoint)
    print ('Latest checkpoint restored!!')

#### Training steps

In [ ]:
@tf.function
def train_step(inp, tar):
    tar_inp = tar[:, :-1]
    tar_real = tar[:, 1:]

    enc_padding_mask, combined_mask, dec_padding_mask = create_masks(inp, tar_inp)

    with tf.GradientTape() as tape:
        predictions, _ = transformer(
            inp, tar_inp, 
            True, 
            enc_padding_mask, 
            combined_mask, 
            dec_padding_mask
        )
        loss = loss_function(tar_real, predictions)

    gradients = tape.gradient(loss, transformer.trainable_variables)    
    optimizer2.apply_gradients(zip(gradients, transformer.trainable_variables))

    train_loss(loss)

In [ ]:
# for epoch in range(EPOCHS):
#     start = time.time()

#     train_loss.reset_states()
  
#     for (batch, (inp, tar)) in enumerate(dataset):
#         train_step(inp, tar)
    
#         # 55k samples
#         # we display 3 batch results -- 0th, middle and last one (approx)
#         # 55k / 64 ~ 858; 858 / 2 = 429
#         if batch % 429 == 0:
#             print ('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1, batch, train_loss.result()))
      
#     if (epoch + 1) % 5 == 0:
#         ckpt_save_path = ckpt_manager.save()
#         print ('Saving checkpoint for epoch {} at {}'.format(epoch+1, ckpt_save_path))

#     # save best model til now

    
#     print ('Epoch {} Loss {:.4f}'.format(epoch + 1, train_loss.result()))

#     print ('Time taken for 1 epoch: {} secs\n'.format(time.time() - start))



### Inference

#### Predicting one word at a time at the decoder and appending it to the output; then taking the complete sequence as an input to the decoder and repeating until maxlen or stop keyword appears

In [ ]:
def evaluate(input_document):
    input_document = document_tokenizer.texts_to_sequences([input_document])
    input_document = tf.keras.preprocessing.sequence.pad_sequences(input_document, maxlen=max_len_news, 
                                                                           padding='post', truncating='post')
    
    encoder_input = tf.expand_dims(input_document[0], 0)

    decoder_input = [summary_tokenizer.word_index[start_token]]
    output = tf.expand_dims(decoder_input, 0)
    
    for i in range(max_len_summary):
        enc_padding_mask, combined_mask, dec_padding_mask = create_masks(encoder_input, output)

        predictions, attention_weights = transformer(
            encoder_input, 
            output,
            False,
            enc_padding_mask,
            combined_mask,
            dec_padding_mask
        )

        predictions = predictions[: ,-1:, :]
        predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)
        # stop prediciting if it reached end_token
        if predicted_id == summary_tokenizer.word_index[end_token]:
            return tf.squeeze(output, axis=0), attention_weights

        output = tf.concat([output, predicted_id], axis=-1)
    return tf.squeeze(output, axis=0), attention_weights

def summarize(input_document):
    # not considering attention weights for now, can be used to plot attention heatmaps in the future
    summarized = evaluate(input_document=input_document)[0].numpy()
    summarized = np.expand_dims(summarized[1:], 0)  # remove start_token
    return summary_tokenizer.sequences_to_texts(summarized)[0]  # since there is just one translated document

#yo bert tokenizer ko lagi xuttai Evaluation function

In [ ]:

# def evaluate(input_document):
#     input_document = document_tokenizer.tokenize(input_document)
#     input_document = tf.keras.preprocessing.sequence.pad_sequences(input_document, maxlen=encoder_maxlen, padding='post', truncating='post')

#     encoder_input = tf.expand_dims(input_document[0], 0)

#     decoder_input = [summary_tokenizer.word_index["<go>"]]
#     output = tf.expand_dims(decoder_input, 0)
    
#     for i in range(decoder_maxlen):
#         enc_padding_mask, combined_mask, dec_padding_mask = create_masks(encoder_input, output)

#         predictions, attention_weights = transformer(
#             encoder_input, 
#             output,
#             False,
#             enc_padding_mask,
#             combined_mask,
#             dec_padding_mask
#         )

#         predictions = predictions[: ,-1:, :]
#         predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)

#         if predicted_id == summary_tokenizer.word_index["<stop>"]:
#             return tf.squeeze(output, axis=0), attention_weights

#         output = tf.concat([output, predicted_id], axis=-1)

#     return tf.squeeze(output, axis=0), attention_weights


mathi ko le kaam garena tei vayera naya halera try gareko tala ko le

In [ ]:

# def evaluate(inp_sentence):
#   # start_token = [document_tokenizer.vocab_size]
#   # end_token = [document_tokenizer.vocab_size + 1]

#   # inp sentence is portuguese, hence adding the start and end token
#   print(inp_sentence)
#   inp_sentence =  document_tokenizer.encode(inp_sentence)
#   print(inp_sentence)
#   encoder_input = tf.expand_dims(inp_sentence, 0)
#   # encoder_input=encoder_vocab_size

#   # as the target is english, the first word to the transformer should be the
#   # english start token.
#   decoder_input = [decoder_vocab_size-1]
#   output = tf.expand_dims(decoder_input, 0)


#   for i in range(40):
#     enc_padding_mask, combined_mask, dec_padding_mask = create_masks(
#         encoder_input, output)

#     # predictions.shape == (batch_size, seq_len, vocab_size)
#     predictions, attention_weights = transformer(encoder_input, 
#                                                  output,
#                                                  False,
#                                                  enc_padding_mask,
#                                                  combined_mask,
#                                                  dec_padding_mask)

#     # # select the last word from the seq_len dimension
#     # predictions = predictions[: ,-1:, :]  # (batch_size, 1, vocab_size)

#     # predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)

#     # # return the result if the predicted_id is equal to the end token
#     # if predicted_id == decoder_vocab_size+1:
#     #   return tf.squeeze(output, axis=0), attention_weights

#     # concatentate the predicted_id to the output which is given to the decoder
#     # as its input.
#     # output = tf.concat([output, predicted_id], axis=-1)
#     print("\n")
#     print(predictions)
#     output=predictions
#     print(f" output is {output}")

  

#   return  output 


In [ ]:
# def summarize(sentence, plot=''):
#   result= evaluate(sentence)

#   predicted_sentence = summary_tokenizer.decode([i for i in result 
#                                             if i < summary_tokenizer.vocab_size])  

#   print('Input: {}'.format(sentence))
#   print('Predicted translation: {}'.format(predicted_sentence))

#   if plot:
#     plot_attention_weights(attention_weights, sentence, result, plot)

In [ ]:
# def summarize(input_document):
#     # not considering attention weights for now, can be used to plot attention heatmaps in the future
#     summarized = evaluate(input_document=input_document)[0].numpy()
#     # summarized = np.expand_dims(summarized[1:], 0)  # not printing <go> token
#     return summary_tokenizer.sequences_to_texts(summarized)[0]  # since there is just one translated document

#bert evaluate yaha bhanda mathi

In [ ]:
def summarize(input_document):
    # not considering attention weights for now, can be used to plot attention heatmaps in the future
    summarized = evaluate(input_document=input_document)[0].numpy()
    summarized = np.expand_dims(summarized[1:], 0)  # not printing <go> token
    return summary_tokenizer.sequences_to_texts(summarized)[0]  # since there is just one translated document

In [ ]:
def validate():
    print('validation started ...')
    val_loss.reset_states()
    for (batch, (inp, tar)) in enumerate(dataset_val):    
        tar_inp = tar[:, :-1] # <startseq> hi im moein
        tar_real = tar[:, 1:] # hi im moein <endseq>

        enc_padding_mask, combined_mask, dec_padding_mask = create_masks(inp, tar_inp)

        # Operations are recorded if they are executed within this context manager
        # and at least one of their inputs is being "watched". Trainable variables are automatically watched
        predictions, _ = transformer(
            inp, tar_inp, 
            False, 
            enc_padding_mask, 
            combined_mask, 
            dec_padding_mask
        )
        loss = loss_function(tar_real, predictions)
        val_loss(loss)
    print('\n* Validation loss: {} '.format(val_loss.result()) )
    return val_loss.result()

In [ ]:
history={'val':[],'train':[]}
EPOCHS = 30
not_progressing = 0
# Computes the (weighted) mean of the given loss values.
train_loss = tf.keras.metrics.Mean(name='train_loss')
val_loss = tf.keras.metrics.Mean(name='val_loss')

In [ ]:

params = {
'lmbda_l2' : lmbda_l2,
'd_out_rate' :d_out_rate,
'num_layers' : num_layers ,
'd_model' : d_model  ,
'dff' : dff ,
'num_heads' : num_heads,
'init_lr':init_lr}
params

{'d_model': 300,
 'd_out_rate': 0.1,
 'dff': 512,
 'init_lr': 0.001,
 'lmbda_l2': 0.1,
 'num_heads': 5,
 'num_layers': 4}

In [ ]:
ep = 1
best_val_loss = np.inf
i1,i2,i3,i4 = np.random.randint(len(summary_val)),np.random.randint(len(summary_val)),np.random.randint(len(summary_val)),np.random.randint(len(summary_val))

In [ ]:
print(params)
print('#'*40)

for epoch in range(ep,EPOCHS+1):
    ep = epoch
    start = time.time()

    train_loss.reset_states()
  
    for (batch, (inp, tar)) in enumerate(dataset):
        
        train_step(inp, tar)
    
        if batch % 150 == 0:
            print ('Epoch {} Batch {} Loss {:.4f}'.format(epoch , batch, train_loss.result()))
                  
    print()
    #print(summarize(document_val[i1]))
    # print(summarize(document_val[i2]))
    # print(summarize(document_val[i3]))
    # print(summarize(document_val[i4]))
    print()
    
    val_loss_ = validate().numpy()
    history['val'].append((epoch,val_loss_))
    print ('\n* Train Loss {:.4f}'.format(train_loss.result()))
    history['train'].append((epoch,train_loss.result().numpy()))
    
    
    if best_val_loss-val_loss_ > 0.1:
        ckpt_save_path4 = ckpt_manager4.save()
        print ('\nSaving checkpoint for epoch {} at {}'.format(epoch, ckpt_save_path4))  
        best_val_loss = val_loss_
    
    hist(history)
    print('Current Lr: ',optimizer2._decayed_lr('float32').numpy())
    print ('\nTime taken for this epoch: {:.2f} secs\n'.format(time.time() - start))
    print('='*40)

{'lmbda_l2': 0.1, 'd_out_rate': 0.1, 'num_layers': 4, 'd_model': 300, 'dff': 512, 'num_heads': 5, 'init_lr': 0.001}
########################################
Epoch 1 Batch 0 Loss 1.7646


validation started ...


InvalidArgumentError: ignored

In [ ]:
# for epoch in range(EPOCHS):
#     start = time.time()

#     train_loss.reset_states()
  
#     for (batch, (inp, tar)) in enumerate(dataset):
#         train_step(inp, tar)
    
#         # 55k samples
#         # we display 3 batch results -- 0th, middle and last one (approx)
#         # 55k / 64 ~ 858; 858 / 2 = 429
#         if batch % 429 == 0:
#             print ('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1, batch, train_loss.result()))
      
#     # if (epoch + 1) % 5 == 0:
#     #     ckpt_save_path = ckpt_manager.save()
#     #     print ('Saving checkpoint for epoch {} at {}'.format(epoch+1, ckpt_save_path))
    
#     print ('Epoch {} Loss {:.4f}'.format(epoch + 1, train_loss.result()))

#     print ('Time taken for 1 epoch: {} secs\n'.format(time.time() - start))


In [ ]:
hist(history)
params

In [ ]:
print(document_val[i1])
print()
print(summarize(document_val[i1]))

In [ ]:
summarize(
    "तराईमा आएको बाढीका बाबजुत यो वर्ष हालसम्मकै उच्च परिमाणमा धान उत्पादन हुने भएको छ। कृषि विकास मन्त्रालयले यो वर्षको धान उत्पादनले विगतका सबै इतिहास तोड्ने दावी गरेको छ। मन्त्रालयका सहप्रवक्ता शंकर सापकोटाले यो वर्ष हालसम्मकै धेरै धान उत्पादन हुने प्रक्षेपण गरिएको बताए।"
)